<a href="https://colab.research.google.com/github/minaringk/sample/blob/main/%EC%A7%80%EC%97%AD%EA%B2%BD%EC%A0%9C_%EB%AA%A8%EB%8B%88%ED%84%B0%EB%A7%81_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%95%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지역경제 데이터 정제 작업



In [2]:
import pandas as pd

In [6]:
df=pd.read_csv('/content/drive/MyDrive/여수지역경제모니터링/군포/KCB_TYP1_STAT_02.txt', delimiter = '|', encoding='utf-16')
df

,BS_YR_MON,GRID_FLAG,GRID_ID,AGE_CD,JOB_CD,MP00001,MP00001_M,MP00001_F,MP00002,MA00001,...,MD00001,MD00003,ML00004,ML00005,ML00012,ML00013,ML00023,ML00026,ML00027,MS00002
0,202201,0,다사497285,60,1,34,14,20,19584,2534,...,0,0,12,5,11173.0,63108.0,3.0,2.0,0.0,863.0
1,202201,0,다사497286,30,2,5,3,2,12469,2804,...,0,0,2,0,57391.0,0.0,1.0,1.0,0.0,788.0
2,202201,0,다사497286,40,3,24,5,19,0,3035,...,1,1,8,2,16484.0,150644.0,2.0,2.0,0.0,855.0
3,202201,0,다사497286,50,1,45,24,21,29211,4025,...,0,0,13,8,30253.0,68547.0,0.0,0.0,0.0,885.0
4,202201,0,다사497288,20,1,5,3,2,5675,2582,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,621.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388184,202207,0,다사486245,30,1,29,9,20,26564,3577,...,1,0,8,5,68076.0,233598.0,1.0,1.0,0.0,901.0
388185,202207,0,다사486246,30,2,5,4,1,13590,3099,...,0,0,2,1,25828.0,89439.0,0.0,0.0,0.0,858.0
388186,202207,0,다사486246,70,1,5,3,2,10136,2783,...,0,0,1,1,0.0,48133.0,0.0,0.0,0.0,941.0
388187,202207,0,다사486256,50,1,29,14,15,14076,5304,...,1,1,10,9,42850.0,75997.0,1.0,1.0,0.0,874.0


In [9]:
df['GRID_FLAG'].unique()
# 0:100m 격자ID, 1: 신우편번호(국가기초구역번호), 2: 통계청 집계구 코드, 3: 행정동(행정기관)단위: 4: 시도 단위


array([0, 2, 1, 3, 4])

In [10]:
df.columns

Index(['BS_YR_MON', 'GRID_FLAG', 'GRID_ID', 'AGE_CD', 'JOB_CD', 'MP00001',
       'MP00001_M', 'MP00001_F', 'MP00002', 'MA00001', 'MA00002', 'MA00004',
       'MA00005', 'MC00002', 'MC00006', 'MC00007', 'MC00011', 'MC00012',
       'MD00001', 'MD00003', 'ML00004', 'ML00005', 'ML00012', 'ML00013',
       'ML00023', 'ML00026', 'ML00027', 'MS00002'],
      dtype='object')

In [22]:
#격자별 월평균 소득
df2=df[df['GRID_FLAG']==0].groupby(['BS_YR_MON', 'GRID_FLAG', 'GRID_ID', 'AGE_CD']).mean()[['MA00001']].reset_index()
df2=df2.set_index(['BS_YR_MON', 'GRID_ID'])[['AGE_CD', 'MA00001']]
df2

AGE_CD  MA00001
BS_YR_MON GRID_ID                  
202101    다사472270      20   1572.0
          다사472270      40   4769.0
          다사476265      60   2802.0
          다사481251      20   2047.5
          다사481251      30   3655.0
...                    ...      ...
202301    다사522286      30   2593.0
          다사522288      30   2633.5
          다사522289      30   1983.0
          다사522289      40   2310.0
          다사523285      50   2333.0

[103304 rows x 2 columns]

In [50]:
icm=df2[[]].reset_index().groupby(['BS_YR_MON', 'GRID_ID']).sum().reset_index()

tot_icm=df2.reset_index().groupby(['BS_YR_MON', 'GRID_ID']).mean()[['MA00001']]
tot_icm.columns=['tot_icm']
tot_icm.reset_index(inplace=True)
icm=pd.merge(icm, tot_icm, on=['BS_YR_MON', 'GRID_ID'], how='left')

for i in range(20, 80, 10) :
  print(i)
  test=df2[df2['AGE_CD']==i][['MA00001']]
  col='tot'+str(i)+'_icm'
  test.columns=[col]
  test.reset_index(inplace=True)
  icm=pd.merge(icm, test,on=['BS_YR_MON', 'GRID_ID'], how='left')

icm

# tot20_icm=df2[df2['AGE_CD']==20][['MA00001']]
# tot20_icm.columns=['tot20_icm']
# tot20_icm.reset_index(inplace=True)
# icm=pd.merge(icm, tot20_icm,on=['BS_YR_MON', 'GRID_ID'], how='left')

# tot30_icm=df2[df2['AGE_CD']==30][['MA00001']]
# tot30_icm.columns=['tot30_icm']
# tot30_icm.reset_index(inplace=True)
# icm=pd.merge(icm, tot30_icm, on=['BS_YR_MON', 'GRID_ID'], how='left')

# tot40_icm=df2[df2['AGE_CD']==40][['MA00001']]
# tot40_icm.columns=['tot40_icm']
# tot40_icm.reset_index(inplace=True)
# icm=pd.merge(icm, tot40_icm, on=['BS_YR_MON', 'GRID_ID'], how='left')

# tot50_icm=df2[df2['AGE_CD']==50][['MA00001']]
# tot50_icm.columns=['tot50_icm']
# tot50_icm.reset_index(inplace=True)
# icm=pd.merge(icm, tot50_icm, on=['BS_YR_MON', 'GRID_ID'], how='left')

# tot60_icm=df2[df2['AGE_CD']==60][['MA00001']]
# tot60_icm.columns=['tot60_icm']
# tot60_icm.reset_index(inplace=True)
# icm=pd.merge(icm, tot60_icm, on=['BS_YR_MON', 'GRID_ID'], how='left')

# tot70_icm=df2[df2['AGE_CD']==70][['MA00001']]
# tot70_icm.columns=['tot70_icm']
# tot70_icm.reset_index(inplace=True)
# icm=pd.merge(icm, tot70_icm, on=['BS_YR_MON', 'GRID_ID'], how='left')
icm

20
30
40
50
60
70


,BS_YR_MON,GRID_ID,tot_icm,tot20_icm,tot30_icm,tot40_icm,tot50_icm,tot60_icm,tot70_icm
0,202101,다사472270,3170.500000,1572.0,NaN,4769.0,NaN,NaN,NaN
1,202101,다사476265,2802.000000,NaN,NaN,NaN,NaN,2802.0,NaN
2,202101,다사481251,3382.600000,2047.5,3655.0,4409.5,3955.0,2846.0,NaN
3,202101,다사481252,2979.666667,2053.0,2790.0,3907.0,4181.0,3235.0,1712.0
4,202101,다사481253,3688.500000,1547.0,3835.0,3874.0,5498.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
20425,202301,다사522284,2118.000000,NaN,2118.0,NaN,NaN,NaN,NaN
20426,202301,다사522286,2593.000000,NaN,2593.0,NaN,NaN,NaN,NaN
20427,202301,다사522288,2633.500000,NaN,2633.5,NaN,NaN,NaN,NaN
20428,202301,다사522289,2146.500000,NaN,1983.0,2310.0,NaN,NaN,NaN


In [37]:
tot70_icm

tot70_icm
BS_YR_MON GRID_ID              
202101    다사481252  1712.000000
          다사481259  2278.666667
          다사481261  3025.500000
          다사482252  1860.000000
          다사482253  1799.000000
...                         ...
202301    다사518277  1907.000000
          다사519274  2049.000000
          다사519276  2444.000000
          다사519277  1877.500000
          다사519278  1887.500000

[14391 rows x 1 columns]

In [52]:
git remote add origin git@github.com:ai-service2023/monitering.git
git branch -M main
git push -u origin main

SyntaxError: ignored